# Record Linkage

## Table of Contents

- [Initialization](#Initialization) 
- [Introduction](#Introduction)
- [Data Definition](#Data-Definition)
- [Parsing](#Parsing)
- [Regular Expressions](#Regular-Expressions)
    - [Exercise 1](#Exercise-1)
- [String Comparators](#String-Comparators)
- [Fellegi-Sunter Record Linkage](#Fellegi-Sunter-Record-Linkage)
    - [Exercise 2](#Exercise-2)
- [References & Further Readings](#References-&-Further-Readings)

## Initialization

- Back to the [Table of Contents](#Table-of-Contents)

Before we start today's class, we need to get everything prepared. Please run the following chunk of code:

In [ ]:
# Importing the modules required in this workbook
import MySQLdb
import numpy as np
import pandas as pd
import re
import jellyfish as jf
import math

## Introduction

- Back to the [Table of Contents](#Table-of-Contents)

In this lesson we will learn a little about how to use Python for cleaning input data, including using regular expressions, as well as the basic idea behind probabilistic record linkage. These two topics fit together naturally, because data cleaning can have a signficant impact on the success of record linkage. Being able to compare fields that were normalized the same way will give better results.

We will use two datasets for the exercises in this chapter. The first will be a list of NIH projects and researchers pulled from the class database. The second is a list of university employees that was scraped from public web sites. 

## Data Definition

- Back to the [Table of Contents](#Table-of-Contents)

Before we begin the task of record linkage, it's important that we understand the variables in our data. In this workbook, we will take a cursory look at some of the values in our data and compute some simple statistics to ensure that the content makes sense. 

Begin by loading the two data sets into pandas data frames. The first file uses tabs as field separators, and we specify this in the `read_csv` method. The second data set contains a zip code field that pandas will try to interpret as a number by default. To prevent this, we explicity set the numpy dtype (data type) that we want to use for that column. After we load the two data sets, we all the `head` method on the first data set to examine the first few reords.

In [ ]:
# make connections to the database
# please use your own credentials
#user = "your account"
#password = "your password"
#database = "homework"

# invoke the connect() function, passing parameters in variables.
#db = MySQLdb.connect( user = user, passwd = password, db = database )

# output basic database connection info.
#print(db)

# obtain ucpay data directly through the DB connection
# need to be tested
#uc = pandas.read_sql_table(*table_name1*, db#, schema=None, index_col=None, 
                      #coerce_float=True, parse_dates=None, columns=None, chunksize=None
#                     )

# obtain nsf award data in 2010-2012 
# need to be tested
#nsf = pandas.read_sql_table(*table_name2*, db#, schema=None, index_col=None, 
                      #coerce_float=True, parse_dates=None, columns=None, chunksize=None
#                     )


# get the first few records from the UC employee file.
#uc.head()

# Don't forget to close the connection
#db.close()


# specify that the file uses tabs as separators
uc = pd.read_csv("Datasets/ucpay2011.csv", sep="\t")

# the zip code field should be read as a character string
nsf = pd.read_csv("Datasets/nsf_awards_2010-2012.csv", dtype={"ZipCode": np.str})


Here are some initial thoughts about the data:
* The ID field looks like a unique identifier that is specific to the data set. If we thought that we were going to use this identifier to link to other data from the same source, then we might take a loser look to see if the values should be interpreted as numbers or character strings. Ideally, this information would appear in the data documentation.
* We should check to make sure that the year field contains valid values.
* The campus and title fields look like they should be interpreted as finite categorical variables.
* The name field appears to contain some redacted values, probably due to a privacy agreement. We will want to link the subset of valid name fields.

Let's perform some quick summaries of the fields in the data. We'll ignore the numeric columns for now, because we won't be using them for linkage, but a thorough data definition process would ensure that these columns are valid as well.

In [ ]:
# print the distinct values in the year, campus, and title columns
print("Distinct years = ", uc["year"].unique())
print("Distinct campuses = ", uc["campus"].unique())
print("Distinct titles = ", uc["title"].unique())

# There are too many titles to display, so let's get the count
print("Number of distinct titles = ", len(uc["title"].unique()))

# Print the total number of rows and the number of rows with valid name values
print("Total rows = ", len(uc))
name = uc["name"]
print("Rows with valid names = ", len(name[name != "***********"]))


Next we'll take a look at the second data set.

In [ ]:
nsf.head()

Observations:
* There are separate fields for first name and last name
* The data include researchers from universities throughout the US, not just in the UC system
* For UC schools we should be able to create a field corresponding to the campus field in the UC data, but this will require some recoding.

In the next two sections we will address some of the data issues that we've identied. Before moving on, we will update the `uc` data set so that it only contains records with valid name fields.

In [ ]:
uc_name = uc[name != "***********"]
uc_name.head()

## Parsing

- Back to the [Table of Contents](#Table-of-Contents)

Above, we noted that we need to parse the name field in the UC data into first, middle, and last name fields. First, let's see what we can do with the built-in `split` method. By default, the `split` method returns a list of strings obtained by splitting the original string on spaces:

In [ ]:
print("Lorem Ipsum".split())
print("carrot cake".split())
print("Bogart, Humphrey".split())

By default, `split` treats whitespae as delimiting characters, and multiple consecutive whitespace characters are treated like a single delimiter. We can also pass an argument to `split` to be the delimiter. If we set the delimiter explicitly, then multiple delimiters are not combined.

In [ ]:
print("Hi,       Mom!".split())
print("Hi,       Mom".split(","))

# in the following example, the result will include the empty strings between the first and second commas
# and the second and third commas
print("Hi,,,Mom!".split(","))

We can map the `split` function over the values in the UC employee `name` column.

In [ ]:
uc_name["name"].apply(str.split).head()

This is pretty good! Fortunately for us, the UC data includes a space between the last name and the comma separating it from the first name. This is unusual, but it means that the built-in `split` method is sufficient. We can define functions `GetLastName` and `GetFirstName` that will call the `split` method and extrac the appropriate values from the resulting list. We won't worry about middle names, because the NSF data doesn't include them.

It turns out that all name fields in the `uc_name` data frame contain more than one word, even after filtering out the redacted names. Our `GetFirstName` function will have to perform a check to make sure that there is a first name to get.

Note that the following code displays a warning which we can safely ignore.

In [ ]:
def GetFirstName(name):
    parts = name.split()
    
    if len(parts) >= 3:
        return parts[2]
    else:
        return None

def GetLastName(name):
    parts = name.split()
    return parts[0]

uc_name["FirstName"] = uc_name["name"].apply(GetFirstName)
uc_name["LastName"] = uc_name["name"].apply(GetLastName)

# Let's see the result
uc_name[["name", "FirstName", "LastName"]].head(15)

I intentionally showed 15 rows instead of the default 10 so that we could see an example of our simple approach failing. The second to last row in the result contains a last name that comprises several words. Our rule, which extracted the first word to be the "last name" does not give the correct result.

There is a more subtle problem as well. It's likely that the third name, WOO YONG, should not be split into a first name and middle name component. That is, WOO YONG may in fact be the first name of the referenced individual. This format is common for names from some countries, including China and Korea.

Let's redefine the `GetFirstName` and `GetLastName` functions so that the last name consists of everything before the comma and the first name consists of the first word after the comma. This doesn't solve our problem with Chinese and Korean names, but it's a good start.

In [ ]:
def GetFirstName(name):
    parts = name.split(",") ########### need to modify, split by ";" ? will check later 
    
    if len(parts) > 1:
        # The name contains a comma. Take the part after the comma, split on whitespace,
        # and grab the first word
        return parts[1].split()[0]
    else:
        return None

def GetLastName(name):
    parts = name.split(",") ########### need to modify, split by ";" ? will check later 
    
    # Return the first part (everything up to the first comma)
    # call the `strip` method to remove the space between the last name and the comma
    return parts[0].strip()

uc_name["FirstName"] = uc_name["name"].apply(GetFirstName)
uc_name["LastName"] = uc_name["name"].apply(GetLastName)

# Let's see the result
uc_name[["name", "FirstName", "LastName"]].head(15)

## Regular Expressions

- Back to the [Table of Contents](#Table-of-Contents)

We were able to solve our parsing problem using the `split` method. Sometimes we need a more heavy duty way to search and extract text. In this section we introduce the basics of regular expressions, because they are a common approach to text parsing and a powerful one, too.

Regular expressions are a mini-language for searching text. We have already noted that regular expressions are *powerful*, but they can also be *complex*. 

> Some people, when confronted with a problem, think "I know, I'll use regular expressions." Now they have two problems. 

> &mdash; Jamie Zawinski

In the previous section we were trying to extract first and last names from the UC name field:

In [ ]:
uc_name["name"].head()

We used basic python functions to try to split the names into first and last name components. In this section we'll use regular expressions to accomplish this task.

In words, this is how I would explain where to find the last name component in the author field: "starting from the beginning of the line, take all the characters until you see a comma." We can build a regular expression that captures this idea from the following components:
* `^` Matches beginning of the line
* `.` Matches any character
* `+` A modifier that means "match one or more of the preceding expression"
The comma is not a special character in regular expressions, so it will match itself.

In [ ]:
m = re.search(r"^.+,", "Su, Yu-Wen")
m

Two things:
* We used "raw" string syntax `r""` to prevent python from escaping any special characters that might appear in our regular expression.
* If the regular expression matches something in the searched string, then `m` holds a python match-data object. Otherwise, it will be `None`.

To see the matched substring, use the `group` function.

In [ ]:
m.group()

This is almost what we want, but it would be nice if we weren't also capturing the comma. The problem is that we need the comma in our regular expression in order to know where the last name ends. The solution is to use regular expression groups by putting parentheses around parts of the regular expression that we want to refer to later.
* `(...)` Creates a group that can be referred to later.

In [ ]:
m = re.search(r"^(.*),", "Franklin, Benjamin")

print(m.group())
print(m.group(1))

In a match-data object `m`, `m.group(1)` will show you what was matched by the expression between the first set of parentheses,  `m.group(2)` will show you what was matched by the expression between the second set of parentheses, and so on. `m.group()` is a synonym for `m.group(0)` and it always refers to what was matched by the entire regular expression.

Now let's write a regular expression that will match the first name of the employee (that is, the first word after the comma). In addition to the bits of regular expressions we saw above, here are some other useful character classes:
* `\w` Matches a single alphanumeric character
* `\W` Matches anything that is not an alphanumeric character
* `\s` Matches a single whitespace character
* `\S` Matches anything that is not a whitespace character.

In [ ]:
regex = r",\s*(\w+)"
m = re.search(regex, "Eisenhower, Dwight David")
m.group(1)

Regular expressions are a powerful tool and we're barely scratching the surface. (See - [References & Further Reading](#References-&-Further-Readings))

### Exercise 1

- Back to the [Table of Contents](#Table-of-Contents)

Now is the time to practise what we have learned(parsing strings and regular expression in Python). As we already have walked through the example of extracting first name and last name, we should be able to get middle name as well. In the exercise, you will be asked to complete your functions for getting middle name, using `split` and regular expression respectively. 


In [ ]:
# Solution begins
def GetMiddleName_split(name):
    '''
    name: given a person's full name
    '''
    
    # extract the middle name based on its position in the full name
    ## Hint: you can use "join" to unlist in python.

    parts = name.split(",")
    
    if len(parts[1]) > 1:
        # The name contains a comma. Take the part after the comma, split on whitespace,
        # and grab starting from the second word

        # extract the middle name based on its position in the full name
        # We can use "join" to unlist in python.
        if " ".join(parts[1].split()[1:]):
            return " ".join(parts[1].split()[1:])
        else:
            return None

    else:
        return None
    
    
def GetMiddleName_re(name):
    '''
    name: given a person's full name
    '''
    
    # define the regular expression to get the middle name
    regex = r",\s*\w+\s(\w+)"
    # store the re.search outcome
    pattern = re.search(regex, name)
    # you can use groups() to get the matched middle names
    # if Null, the function should return None.
    if pattern:
        return pattern.groups()[0]
    else:
        return None
# End of solution

In [ ]:
# let's see how your functions work
uc_name["MiddleName"] = uc_name["name"].apply(GetMiddleName_split)
uc_name["MiddleName_re"] = uc_name["name"].apply(GetMiddleName_re)
uc_name[["name", "FirstName", "LastName","MiddleName","MiddleName_re"]].head(30)

## String Comparators

- Back to the [Table of Contents](#Table-of-Contents)

In this section we will demonstrate different string comparator algorithms which are provided by the [jellyfish](https://github.com/sunlightlabs/jellyfish) package by writing a function that takes a name as an argument and returns the names in the NSF data that are most similar to the argument. We will start by importing the package and creating a `set` of unique first names from the NSF data. The `FistName` field is missing some values which are represented as NaN in the data frame. To prevent errors later on, we only include valid character strings (which have type `str`) in our list of unique names.

In [ ]:
# make a set storing the unique first name with respect to the nsf dataset
uniq_first_names = set(name for name in nsf["FirstName"] if type(name) == str)

Next, we will define our function. It should:
* Take a name (i.e., a string) as an argument
* Optionally, take a number `n` which determines the size of the output
* Compare the name to each name in `uniq_first_names` using the Levenshtein distance string comparator
* Return a list of the `n` names in `uniq_first_names` that are closest to the argument

Here's one implementation. Note that in the comparison we capitalize both names being compared, so that letter case doesn't affect the final distance.

In [ ]:
def closest_names(name, n=10):
    # First create a list of tuples (other_name, distance), where
    # other_name is taken from uniq_first_names
    distances = [(other_name, jf.levenshtein_distance(unicode(name.upper()), unicode(other_name.upper()))) 
                 for other_name in uniq_first_names]
    
    # Sort distances by the second element in the tuple, and return the top n values
    return sorted(distances, key=lambda x: x[1])[:n]

Let's try it out on some names.

In [ ]:
# experiment the function with several names.
print(closest_names("Jennifer"))
print(closest_names("Sonya"))
print(closest_names("Wai Tong"))

Recall that Levenshtein distance is a kind of edit distance. Edit distances count the number of edit operations needed to change one word to another, and different edit distances count different edit operations as valid. In the case of Levenshtein distance, the valid edit operations are inserting a letter, deleting a letter, or changing a letter. 

It would be interesting to compare this output to the output from other string comparators included in the jellyfish package:
* Levenshtein distance: edit distance where the valid operations are inserting a letter, deleting a letter, or changing a letter
* Levenshtein-Damerau distance: edit distance which includes the same operations as Levenshtein distance but also allows transposing two adjacent letters. This can be useful for finding words with typos.
* Jaro-Winkler distance: a fast-to-compute string distance based on common letters between two words

Let's update our `closest_names` function so that we can specify the string comparator we want to use. Note that for edit distance smaller numbers indicate closer strings, but for Jaro-Winkler distance larger values indicate closer strings. We will have to add an option to our new function that lets us specify the sort order that determines which strings are close.

In [ ]:
def closest_names_2(string_comparator, name, reverse_sort=False, n=10):
    # First create a list of tuples (other_name, distance), where
    # other_name is taken from uniq_first_names
    distances = [(other_name, string_comparator(unicode(name.upper()), unicode(other_name.upper())))
                 for other_name in uniq_first_names]
    
    # Sort distances by the second element in the tuple, and return the top n values
    return sorted(distances, key=lambda x: x[1], reverse=reverse_sort)[:n]

In [ ]:
# Try it!
print(closest_names_2(jf.damerau_levenshtein_distance, "William"))
print(closest_names_2(jf.jaro_winkler, "Wiliam", reverse_sort=True))

## Fellegi-Sunter Record Linkage

- Back to the [Table of Contents](#Table-of-Contents)

In this section we will "manually" perform the steps in Fellegi-Sunter record linkage. Our goal is to illustrate the Fellegi-Sunter algorithm by breaking it into bitesize pieces. 

In our example we will compare first names and last names using Levenshtein distance. In the Fellegi-Sunter algorithm, the result of a field comparison is assumed to follow a multinomial distribution. That means it can only take on finitely many values. Therefore we will define a function that compares to strings and returns the value 2, 1, or 0 to indicate an exact match, a nearly exact match, or anything else.

In [ ]:
# define a function that tells us how different two input strings are.
def fuzzy_string_comparator(s1, s2):
    '''
    s1: input string No.1
    s2: input string No.2
    '''
    # check if they are all strings
    if type(s1) != str or type(s2) != str:
        return 0
    # calculate Jaro-Winkler distance after converting two strings into capital characters.
    dist = jf.jaro_winkler(unicode(s1.upper()), unicode(s2.upper()))
    # according different thresholds, return the match level
    if dist >= 0.92:
        return 2
    elif dist >= 0.85:
        return 1
    else:
        return 0
    


In [ ]:
# let's see how the fuzzy_string_comparator work
print(fuzzy_string_comparator("joshua", "joshua"))
print(fuzzy_string_comparator("joshua", "joshau"))
print(fuzzy_string_comparator("joshua", "todd"))

The above function compares *fields* in a record. Next, we define a function that compares *records*. My record comparison function assumes that records will have the form of a tuple: (identifier, first name, last name). It returns a length 2 tuple that gives the result of applying our fuzzy string comparator to the first name and to the last name.

In [ ]:
def comparison_vector(rec1, rec2):
    '''
    rec1: input record No.1
    rec2: input record No.2
    '''
    #rec1 and rec2 have the form (id, first name, last name)
    
    # m, n store the comparing outcomes of first name and last name.
    m = fuzzy_string_comparator(rec1[1], rec2[1])
    n = fuzzy_string_comparator(rec1[2], rec2[2])
    return (m, n)



In [ ]:
# try out the comparison_vector function
print(comparison_vector((1, "joshua", "tokle"), (2, "joshua", "smith")))
print(comparison_vector((3, "joshua", "tokle"), (4, "josue", "tolke")))

Next, we need to define m-weights and u-weights. An m-weight is the probability of seeing a particular field comparison outcome given that we are comparing two records that represent the same individual. A u-weight is the probability of seeing a particular field outcome given that we are comparing two records that do *not* represent the same individual. For example, if two records represent the same person, the first and last names should match with high probability (the m-weights for the comparison value 2 should be large). On the other hand, suppose we had month of birth in our data set. The probability that two random individuals will agree on month of birth is about 1/12, so we would assign a u-weight of about 1/12 to this outcome.

I'm going to make up some m-weights and u-weights here. In practice you might try to fit these to data or tweak them after seeing preliminary output. I don't claim that these are particularly good.

In [ ]:
# The outer list is length 2, corresponding to the number of fields we're comparing.
# Each inner list has length 3, because fuzzy string comparator returns 3 possible values.

m_weights = [[0.01, 0.14, 0.85], # m-weights corresponding to first name
             [0.01, 0.09, 0.90]] # m-weights corresponding to last name

u_weights = [[0.88, 0.10, 0.02], # u-weights corresponding to first name
             [0.91, 0.08, 0.01]] # u-weights corresponding to last name

Now we define a function that compares two records and returns the match score.

In [ ]:
# math_score not only compares 2 records but also gives the log-likelihood-ratio of matchness
def match_score(rec1, rec2):
    '''
    rec1: input record No.1
    rec2: input record No.2
    '''
    # calulate the similarity level using comparison_vector
    m, n = comparison_vector(rec1, rec2)
    
    # use m and n to identify the corresopoing weights    
    
    # what's the log-probability of seeing this comparison vector if the records are a match?
    log_pr_given_match = math.log(m_weights[0][m]) + math.log(m_weights[1][n])
    
    # what's the log-probability of seeing this comparison vector if the records are a nonmatch?
    log_pr_given_nonmatch = math.log(u_weights[0][m]) + math.log(u_weights[1][n])
    
    # return the log-likelihood-ratio
    return log_pr_given_match - log_pr_given_nonmatch



In [ ]:
# have a rough look at its sample output
print(match_score((1, "joshua", "tokle"), (2, "joshua", "smith")))
print(match_score((1, "joshua", "tokle"), (4, "joshu", "tolke")))
print(match_score((1, "joshua", "tokle"), (7, "christina", "jones")))

Finally, we can try to link UC employees to NSF researchers. We will apply **blocking** to reduce the number of comparisons we perform. In particular, I am only going to compare graduate students whose last name begins with the letter "H". I will also limit the NSF researchers to those located in California.

In [ ]:
# subset the uc data, taking those last names start with "H"
startswith_h = uc_name["LastName"].apply(lambda s: s[0] == "H")
uc_h = uc_name[startswith_h]

# take Californian nsf records for comparison, also we only look at last names starting with "H"
nsf_ca = nsf[(nsf['StateCode'] == "CA")]
startswith_h = nsf_ca["LastName"].apply(lambda s: type(s) == str and s[0].upper() == "H")
nsf_h = nsf_ca[startswith_h]

In [ ]:
# This cell takes a minute to execute

# create an empty list to save the outputs
potential_matches = []


for _, uc_row in uc_h.iterrows():
    rec1 = (uc_row["ID"], uc_row["FirstName"], uc_row["LastName"])
    for nsf_ix, nsf_row in nsf_h.iterrows():
        rec2 = (nsf_ix, nsf_row["FirstName"], nsf_row["LastName"])
        score = match_score(rec1, rec2)
        if score >= 0.5:
            potential_matches.append((score, rec1, rec2))
            
# Sort the output so the best matches appear at the top
potential_matches = sorted(potential_matches, key=lambda x: x[0], reverse=True)
potential_matches

# How did we do?

### Exercise 2

- Back to the [Table of Contents](#Table-of-Contents)

Let's practice record linkage on other sample datasets. In this section, you will be asked to implement the previouly defined functions hence have a better understanding of applying Probabilistic Record Linkage. 


In [ ]:
'''
    Please compare the following pairs of strings using appropriate function:
    "Jellyfish" vs "Smellyfish"
    "jellyfihs" vs "smellyfihs"
    "Jellyfish" vs "Rib-eye Steak"
    
'''

# Solution begins
print(fuzzy_string_comparator("Jellyfish", "Smellyfish"))
print(fuzzy_string_comparator("jellyfihs", "smellyfihs"))
print(fuzzy_string_comparator("Jellyfish", "Rib-eye Steak"))

# End of solution
    

In [ ]:
'''
    Please compare the following pairs of records using appropriate function:
    (1, "Jelly", "fish") vs (1, "Smelly", "fish")
    (3, "jelly", "fihs") vs (4, "smelly", "dish")
    (5, "Jelly", "fish") vs (18, "Rib-eye", "Steak")
    
'''

# Solution begins
print(comparison_vector((1, "Jelly", "fish"), (1, "Smelly", "fish")))
print(comparison_vector((3, "jelly", "fihs"), (4, "smelly", "dish")))
print(comparison_vector((5, "Jelly", "fish"), (18, "Rib-eye", "Steak")))

# End of solution

## References & Further Readings

- Back to the [Table of Contents](#Table-of-Contents)

### Parsing

* Python online ducumentation: https://docs.python.org/2/library/string.html#deprecated-string-functions
* Python 2.7 Tutorial(Splitting and Joining Strings): http://www.pitt.edu/~naraehan/python2/split_join.html

### Regular Expression

* Python documentation: https://docs.python.org/2/library/re.html#regular-expression-syntax
* Online regular expression tester (good for learning): http://regex101.com/

### String Comparators

* GitHub page of jellyfish: https://github.com/jamesturk/jellyfish
* Different distances that measure the differences between strings:
    - Levenshtein distance: https://en.wikipedia.org/wiki/Levenshtein_distance
    - Damerau–Levenshtein distance: https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance
    - Jaro–Winkler distance: https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance
    - Hamming distance: https://en.wikipedia.org/wiki/Hamming_distance
    - Match rating approach: https://en.wikipedia.org/wiki/Match_rating_approach

### Fellegi-Sunter Record Linkage 

* Introduction to Probabilistic Record Linkage: http://www.bristol.ac.uk/media-library/sites/cmm/migrated/documents/problinkage.pdf
* Paper Review: https://www.cs.umd.edu/class/spring2012/cmsc828L/Papers/HerzogEtWires10.pdf

